In [91]:
# Célula 1: Importar as bibliotecas necessárias
import pandas as pd
import numpy as np
import os
import openpyxl
import xlsxwriter
import glob



In [92]:
# Função: Carregamento de dados CSV/XLSX
caminho_pasta = '../../dados_brutos/PNAD_2009'

agressao = glob.glob(os.path.join(caminho_pasta, 'agressao/*.xls'), recursive=True)
furto = glob.glob(os.path.join(caminho_pasta, 'furto/*.xls'), recursive=True)
roubo = glob.glob(os.path.join(caminho_pasta, 'roubo/*.xls'), recursive=True)
roubofurto = glob.glob(os.path.join(caminho_pasta, 'roubofurto/*.xls'), recursive=True)
seguranca = glob.glob(os.path.join(caminho_pasta, 'seguranca/*.xls'), recursive=True)
tentativa = glob.glob(os.path.join(caminho_pasta, 'tentativa/*.xls'), recursive=True)


def processar_arquivo(tipo):
    dados = {}

    for arquivo in tipo:
        try:
            nome = os.path.basename(arquivo).replace('.xlsx', '').replace('.xls', '')
            xls = pd.ExcelFile(arquivo)
            for sheet_name in xls.sheet_names:
                df_nome = f"{nome}_{sheet_name}" if len(xls.sheet_names) > 1 else nome
                dados[df_nome] = pd.read_excel(arquivo, sheet_name=sheet_name, header=[0, 1], skiprows=7, index_col=0, skipfooter=3)


                # tratamento dos dados

                dados[df_nome].rename(columns={
                    'unnamed: 1_level_1': '',
                    'Preta ou parda ': 'preta/parda',
                    'Preta ou parda .1': 'preta',
                    'Preta ou parda .2': 'parda',
                    'Sem rendimento a menos de 1/4 (2)': 'sem rendimento a menos de 1/4',
                    'sem rendimento a menos de 1/4 (2)'
                    'Unnamed: 5_level_1': 'total'
                }, inplace=True)

                if isinstance(dados[df_nome].columns, pd.MultiIndex):
                    dados[df_nome].columns = pd.MultiIndex.from_tuples([
                        tuple(str(c).replace('\n', '').strip() for c in col)
                        for col in dados[df_nome].columns
                    ])
                else:
                    dados[df_nome].columns = dados[df_nome].columns.str.replace('\n', '', regex=True).str.strip()


                dados[df_nome] = dados[df_nome][dados[df_nome].index.notna()]
                
                dados[df_nome].index = dados[df_nome].index.astype(str).str.lower()
                dados[df_nome].columns = pd.MultiIndex.from_tuples([tuple(str(item).lower() for item in col) for col in dados[df_nome].columns])

                for col in dados[df_nome].select_dtypes(include='object').columns:
                    dados[df_nome][col] = dados[df_nome][col].str.lower()

                dados[df_nome] = dados[df_nome].apply(pd.to_numeric, errors='coerce')
                dados[df_nome] = dados[df_nome].round(2)


                

    

                


                print(f"[OK] Excel {df_nome}")
        except Exception as e:
            print(f"[ERRO] Excel {arquivo}: {e}")

    return dados

dados_agressao = processar_arquivo(agressao)
dados_furto = processar_arquivo(furto)
dados_roubo = processar_arquivo(roubo)
dados_roubofurto = processar_arquivo(roubofurto)
dados_seguranca = processar_arquivo(seguranca)
dados_tentativa = processar_arquivo(tentativa)


[OK] Excel cv126011a
[OK] Excel cv126012a
[OK] Excel cv126013a
[OK] Excel cv126021a
[OK] Excel cv126022a
[OK] Excel cv126023a
[OK] Excel cv126031a
[OK] Excel cv126032a
[OK] Excel cv126033a
[OK] Excel cv126041a
[OK] Excel cv126042a
[OK] Excel cv126051a
[OK] Excel cv126052a
[OK] Excel cv126061a
[OK] Excel cv126062a
[ERRO] Excel ../../dados_brutos/PNAD_2009\agressao\cv126071a.xls: Can only use .str accessor with string values!
[ERRO] Excel ../../dados_brutos/PNAD_2009\agressao\cv126072a.xls: Can only use .str accessor with string values!
[ERRO] Excel ../../dados_brutos/PNAD_2009\agressao\cv126081a.xls: Can only use .str accessor with string values!
[ERRO] Excel ../../dados_brutos/PNAD_2009\agressao\cv126082a.xls: Can only use .str accessor with string values!
[OK] Excel cv126091a
[OK] Excel cv126092a
[OK] Excel cv126101a
[ERRO] Excel ../../dados_brutos/PNAD_2009\agressao\cv126102a.xls: Can only use .str accessor with string values!
[ERRO] Excel ../../dados_brutos/PNAD_2009\agressao\cv1261

In [93]:
print(dados_agressao['cv126011a'].columns.tolist())

[('total(1)', 'unnamed: 1_level_1'), ('sexo', 'homens'), ('sexo', 'mulheres'), ('cor ou raça', 'branca'), ('cor ou raça', 'preta/parda')]


In [94]:
display(dados_roubofurto['cv123011a'].info())
display(dados_roubofurto['cv123011a'].head())

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries,                brasil to distrito federal
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   (total(1), unnamed: 1_level_1)  35 non-null     float64
 1   (sexo, homens)                  35 non-null     float64
 2   (sexo, mulheres)                35 non-null     float64
 3   (cor ou raça, branca)           35 non-null     float64
 4   (cor ou raça, preta/parda)      35 non-null     object 
 5   (cor ou raça, preta)            35 non-null     object 
 6   (cor ou raça, parda)            35 non-null     object 
dtypes: float64(4), object(3)
memory usage: 2.3+ KB


None

total(1)      sexo  \
                                        unnamed: 1_level_1    homens   
               brasil                             1.227527  1.285665   
situação do domicílio                                  NaN       NaN   
urbana                                            1.333160  1.427198   
rural                                             4.992342  4.968521   
grandes regiões e unidades da federação                NaN       NaN   

                                                  cor ou raça              \
                                         mulheres      branca preta/parda   
               brasil                    1.505970    1.553339    1.590327   
situação do domicílio                         NaN         NaN         NaN   
urbana                                   1.564227    1.637694    1.727484   
rural                                    6.694034    6.382846    6.086938   
grandes regiões e unidades da federação       NaN         NaN         NaN   

                                                              
                                             preta     parda  
               brasil                     3.094999  1.643328  
situação do domicílio                          NaN       NaN  
urbana                                    3.211566  1.797979  
rural                                    12.651894  6.140072  
grandes regiões e unidades da federação        NaN       NaN

In [95]:
display(dados_agressao['cv126011a'].info())
display(dados_agressao['cv126011a'].head())


<class 'pandas.core.frame.DataFrame'>
Index: 37 entries,                brasil to distrito federal
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   (total(1), unnamed: 1_level_1)  35 non-null     float64
 1   (sexo, homens)                  35 non-null     float64
 2   (sexo, mulheres)                35 non-null     float64
 3   (cor ou raça, branca)           35 non-null     float64
 4   (cor ou raça, preta/parda)      35 non-null     float64
dtypes: float64(5)
memory usage: 1.7+ KB


None

total(1)   sexo           \
                                        unnamed: 1_level_1 homens mulheres   
               brasil                                 2.18   2.52     2.75   
situação do domicílio                                  NaN    NaN      NaN   
urbana                                                2.33   2.68     2.91   
rural                                                 7.28   8.16     9.43   
grandes regiões e unidades da federação                NaN    NaN      NaN   

                                        cor ou raça              
                                             branca preta/parda  
               brasil                          2.92        2.66  
situação do domicílio                           NaN         NaN  
urbana                                         3.09        2.86  
rural                                         10.66        8.11  
grandes regiões e unidades da federação         NaN         NaN

In [96]:
todos_dados = {
    'agressao': dados_agressao,
    'furto': dados_furto,
    'roubo': dados_roubo,
    'roubofurto': dados_roubofurto,
    'seguranca': dados_seguranca,
    'tentativa': dados_tentativa
}

display(todos_dados['agressao']['cv126011a'].info())
display(todos_dados['agressao']['cv126011a'].head())

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries,                brasil to distrito federal
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   (total(1), unnamed: 1_level_1)  35 non-null     float64
 1   (sexo, homens)                  35 non-null     float64
 2   (sexo, mulheres)                35 non-null     float64
 3   (cor ou raça, branca)           35 non-null     float64
 4   (cor ou raça, preta/parda)      35 non-null     float64
dtypes: float64(5)
memory usage: 1.7+ KB


None

total(1)   sexo           \
                                        unnamed: 1_level_1 homens mulheres   
               brasil                                 2.18   2.52     2.75   
situação do domicílio                                  NaN    NaN      NaN   
urbana                                                2.33   2.68     2.91   
rural                                                 7.28   8.16     9.43   
grandes regiões e unidades da federação                NaN    NaN      NaN   

                                        cor ou raça              
                                             branca preta/parda  
               brasil                          2.92        2.66  
situação do domicílio                           NaN         NaN  
urbana                                         3.09        2.86  
rural                                         10.66        8.11  
grandes regiões e unidades da federação         NaN         NaN

In [97]:
def exportar_dados_organizados(dicionario_principal, pasta_base='../../dados_tratados/PNAD_2009'):
    """
    Exporta um dicionário aninhado de DataFrames para arquivos CSV e XLSX,
    organizando os arquivos em pastas por categoria e por formato.
    """
    print(f"Iniciando a exportação para a pasta base: '{pasta_base}'")
    
    # Verifica se o dicionário principal não está vazio
    if not dicionario_principal:
        print("[AVISO] O dicionário de dados está vazio. Nenhuma exportação será feita.")
        return

    # Loop pelas categorias (ex: 'agressao', 'furto')
    for categoria, dicionario_dataframes in dicionario_principal.items():
        print(f"\n--- Exportando categoria: {categoria.upper()} ---")
        
        # Define e cria os diretórios de saída para esta categoria
        path_saida_csv = os.path.join(pasta_base, 'csv', categoria)
        path_saida_xlsx = os.path.join(pasta_base, 'xlsx', categoria)
        os.makedirs(path_saida_csv, exist_ok=True)
        os.makedirs(path_saida_xlsx, exist_ok=True)
        
        # Loop pelos DataFrames dentro da categoria
        for nome_df, df in dicionario_dataframes.items():
            
            # --- Exportação para CSV ---
            # Define o caminho completo do arquivo
            caminho_csv = os.path.join(path_saida_csv, f"{nome_df}.csv")
            try:
                # sep=';' e encoding='utf-8-sig' são ideais para abrir no Excel em português
                df.to_csv(caminho_csv, sep=';', encoding='utf-8-sig', index=True)
                print(f"  [OK] CSV salvo: {caminho_csv}")
            except Exception as e:
                print(f"  [ERRO] Falha ao salvar CSV {caminho_csv}: {e}")

            # --- Exportação para XLSX ---
            # Define o caminho completo do arquivo
            caminho_xlsx = os.path.join(path_saida_xlsx, f"{nome_df}.xlsx")
            try:
                # index=True garante que o índice do DataFrame seja salvo
                df.to_excel(caminho_xlsx, index=True, sheet_name='dados')
                print(f"  [OK] XLSX salvo: {caminho_xlsx}")
            except Exception as e:
                print(f"  [ERRO] Falha ao salvar XLSX {caminho_xlsx}: {e}")
                
    print(f"\n[SUCESSO] Exportação de todos os dados concluída!")
    print(f"Verifique a pasta '{pasta_base}' no seu diretório.")


In [98]:
def exportar_categorias_em_abas(dados_aninhados, pasta_saida='../../dados_tratados/PNAD_2009'):
    """
    Exporta cada categoria para um único arquivo Excel, onde cada DataFrame
    da categoria se torna uma aba (planilha) dentro do arquivo.
    """
    print(f"Iniciando a exportação por categoria para a pasta: '{pasta_saida}'")
    
    # Cria a pasta de saída se ela não existir
    os.makedirs(pasta_saida, exist_ok=True)

    # Loop pelas categorias (ex: 'agressao', 'furto')
    for categoria, dicionario_dataframes in dados_aninhados.items():
        # Define o caminho do arquivo Excel para a categoria atual
        caminho_arquivo_xlsx = os.path.join(pasta_saida, f"{categoria}.xlsx")
        
        print(f"\n--- Criando arquivo para a categoria: {categoria.upper()} ---")
        print(f"Arquivo: {caminho_arquivo_xlsx}")

        try:
            # Usa o ExcelWriter para escrever várias abas no mesmo arquivo
            with pd.ExcelWriter(caminho_arquivo_xlsx, engine='xlsxwriter') as writer:
                # Loop pelos DataFrames dentro da categoria
                for nome_aba, df in dicionario_dataframes.items():
                    # Escreve cada DataFrame em uma aba com seu respectivo nome
                    # 'index=True' garante que a coluna de índice seja salva
                    df.to_excel(writer, sheet_name=nome_aba, index=True)
                    print(f"  -> Aba '{nome_aba}' salva.")
        
        except Exception as e:
            print(f"[ERRO] Falha ao criar o arquivo para a categoria '{categoria}': {e}")
            
    print(f"\n[SUCESSO] Exportação por categoria concluída!")
    print(f"Verifique a pasta '{pasta_saida}'.")


In [99]:
exportar_dados_organizados(todos_dados)

Iniciando a exportação para a pasta base: '../../dados_tratados/PNAD_2009'

--- Exportando categoria: AGRESSAO ---
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\cv126011a.csv
  [OK] XLSX salvo: ../../dados_tratados/PNAD_2009\xlsx\agressao\cv126011a.xlsx
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\cv126012a.csv
  [OK] XLSX salvo: ../../dados_tratados/PNAD_2009\xlsx\agressao\cv126012a.xlsx
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\cv126013a.csv
  [OK] XLSX salvo: ../../dados_tratados/PNAD_2009\xlsx\agressao\cv126013a.xlsx
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\cv126021a.csv
  [OK] XLSX salvo: ../../dados_tratados/PNAD_2009\xlsx\agressao\cv126021a.xlsx
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\cv126022a.csv
  [OK] XLSX salvo: ../../dados_tratados/PNAD_2009\xlsx\agressao\cv126022a.xlsx
  [OK] CSV salvo: ../../dados_tratados/PNAD_2009\csv\agressao\cv126023a.csv
  [OK] XLSX salvo: ../../dados_tra

In [100]:
exportar_categorias_em_abas(todos_dados)

Iniciando a exportação por categoria para a pasta: '../../dados_tratados/PNAD_2009'

--- Criando arquivo para a categoria: AGRESSAO ---
Arquivo: ../../dados_tratados/PNAD_2009\agressao.xlsx
  -> Aba 'cv126011a' salva.
  -> Aba 'cv126012a' salva.
  -> Aba 'cv126013a' salva.
  -> Aba 'cv126021a' salva.
  -> Aba 'cv126022a' salva.
  -> Aba 'cv126023a' salva.
  -> Aba 'cv126031a' salva.
  -> Aba 'cv126032a' salva.
  -> Aba 'cv126033a' salva.
  -> Aba 'cv126041a' salva.
  -> Aba 'cv126042a' salva.
  -> Aba 'cv126051a' salva.
  -> Aba 'cv126052a' salva.
  -> Aba 'cv126061a' salva.
  -> Aba 'cv126062a' salva.
  -> Aba 'cv126071a' salva.
  -> Aba 'cv126072a' salva.
  -> Aba 'cv126081a' salva.
  -> Aba 'cv126082a' salva.
  -> Aba 'cv126091a' salva.
  -> Aba 'cv126092a' salva.
  -> Aba 'cv126101a' salva.
  -> Aba 'cv126102a' salva.
  -> Aba 'cv126111a' salva.
  -> Aba 'cv126112a' salva.
  -> Aba 'cv126121a' salva.
  -> Aba 'cv126122a' salva.
  -> Aba 'cv126131a' salva.
  -> Aba 'cv126132a' salva